In [1]:
#load the 
from timeit import default_timer as timer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import requests
import warnings
from IPython.display import HTML
warnings.filterwarnings('ignore')

### load the dataset



In [36]:
iphone = pd.read_csv ('C:\\Users\\KATHIR VELU\\Downloads\\iphone_purchase_records (1).csv')

In [37]:
iphone.head()

,Gender,Age,Salary,Purchase Iphone
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [38]:
iphone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           400 non-null    object
 1   Age              400 non-null    int64 
 2   Salary           400 non-null    int64 
 3   Purchase Iphone  400 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 12.6+ KB


In [39]:
iphone.describe()

,Age,Salary,Purchase Iphone
count,400.000000,400.000000,400.000000
mean,37.655000,69742.500000,0.357500
std,10.482877,34096.960282,0.479864
min,18.000000,15000.000000,0.000000
25%,29.750000,43000.000000,0.000000
50%,37.000000,70000.000000,0.000000
75%,46.000000,88000.000000,1.000000
max,60.000000,150000.000000,1.000000


In [40]:
iphone.columns

Index(['Gender', 'Age', 'Salary', 'Purchase Iphone'], dtype='object')

### change the data type of the gender column

In [41]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
iphone['Gender']=le.fit_transform(iphone['Gender'])

### Assigning x and y

In [42]:
x=iphone[['Gender','Age','Salary']]
y=iphone['Purchase Iphone']


In [43]:
x.head()

,Gender,Age,Salary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000


In [44]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Purchase Iphone, dtype: int64

### split the data to train and test

In [46]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [47]:
x_train

,Gender,Age,Salary
350,0,38,113000
21,0,47,49000
361,0,53,34000
85,0,31,118000
378,1,41,87000
...,...,...,...
285,0,37,93000
284,1,48,141000
374,0,37,80000
224,0,35,60000


In [48]:
y_train

350    1
21     1
361    1
85     1
378    1
      ..
285    1
284    0
374    0
224    0
289    1
Name: Purchase Iphone, Length: 300, dtype: int64

### patch with intel oneapi with scikit learn

In [49]:
#patch the scikit learn 
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [50]:
from sklearn.linear_model import LogisticRegression

params = {
    'C': 0.1,
    'solver': 'lbfgs',
    'multi_class': 'multinomial',
    'n_jobs': -1,
}
start = timer()
classifier = LogisticRegression(**params).fit(x_train, y_train)
train_patched = timer() - start
f"Intel® extension for Scikit-learn time: {train_patched:.2f} s"

'Intel® extension for Scikit-learn time: 4.43 s'

In [51]:
'Intel® extension for Scikit-learn time: 0.60 s'

'Intel® extension for Scikit-learn time: 0.60 s'

In [52]:
y_predict = classifier.predict_proba(x_test)
log_loss_opt = metrics.log_loss(y_test, y_predict)
f"Intel® extension for Scikit-learn Log Loss: {log_loss_opt} s"

'Intel® extension for Scikit-learn Log Loss: 0.6867957800748017 s'

### unpatch the oneapi scikit learn

In [54]:
#unpatch the oneapi scikit learn 
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [55]:
from sklearn.linear_model import LogisticRegression

start = timer()
classifier = LogisticRegression(**params).fit(x_train, y_train)
train_unpatched = timer() - start
f"Original Scikit-learn time: {train_unpatched:.2f} s"


'Original Scikit-learn time: 14.35 s'

In [56]:
y_predict = classifier.predict_proba(x_test)
log_loss_original = metrics.log_loss(y_test, y_predict)
f"Original Scikit-learn Log Loss: {log_loss_original} s"

'Original Scikit-learn Log Loss: 0.682067178101897 s'

In [57]:
HTML(f"<h3>Compare Log Loss metric of patched Scikit-learn and original</h3>"
     f"Log Loss metric of patched Scikit-learn: {log_loss_opt} <br>"
     f"Log Loss metric of unpatched Scikit-learn: {log_loss_original} <br>"
     f"Metrics ratio: {log_loss_opt/log_loss_original} <br>"
     f"<h3>With Scikit-learn-intelex patching you can:</h3>"
     f"<ul>"
     f"<li>Use your Scikit-learn code for training and prediction with minimal changes (a couple of lines of code);</li>"
     f"<li>Fast execution training and prediction of Scikit-learn models;</li>"
     f"<li>Get the similar quality</li>"
     f"<li>Get speedup in <strong>{(train_unpatched/train_patched):.1f}</strong> times.</li>"
     f"</ul>")